# LLM tool calling

> Disclosure: A DeepSeek-R1 LLM was involved in drafting some of the summarizing text in this tutorial. Where this was the case, it is indicated by a footnote stating the prompt. The tiny class used for queries to the LLM-as-a-service is shown in Appendix&nbsp;A. All other text and code examples were written by a human.

## What is _tool calling_?<sup>1</sup>

**Tool Calling in LLM Queries: A Summary**

1. **Definition**: Tool calling is a technique where Large Language Models (LLMs) interpret user input as commands to invoke external tools, services, or APIs.

2. **Functionality**: It enables models to execute tasks beyond their training data by interacting with databases, cloud services, custom scripts, and other systems in real-time.

3. **Purpose**: Enhances the model's capabilities, allowing it to perform dynamic actions based on user requests.

4. **Benefits**:
   - Extends LLM functionality.
   - Facilitates complex tasks through natural language input.

5. **Challenges**:
   - Potential security risks if inputs aren't properly sanitized.
   - Requires robust error handling and API integration.

This approach bridges the gap between NLP capabilities and external systems, offering versatile applications in automation, data retrieval, and more.

## How is tool calling performed in practice?

The process is best illustrateted with a minimal example.
As usual, we need to connect to the LLM service and make sure to pick a model that _supports tool calling._
To check if a model supports tool calling you can e.g. use the ollama site:
<https://ollama.com/search?c=tools>

In [ ]:
!pip -q install ollama

In [ ]:
from ollama import Client

OLLAMA_HOST = 'http://10.129.20.4:9090'
OLLAMA_MODEL = 'llama3.3:latest'

client = Client(host=OLLAMA_HOST)

In addition we need to provide a _tool_, that can be as simple as the following function:

In [ ]:
def add_numbers(a: int, b: int) -> int:
    """
    Add numbers

    Args:
      a (int): The first number
      b (int): The second number

    Returns:
      int: The sum of the two numbers
    """
    return a + b

Here, the name of the tool is unimportant, but the type annotations and the docstring is. The LLM use python's introspection capabilities to analyze the function, but type annotation and documentation helps in getting the desired result.

We can now proceed to query the LLM:

In [ ]:
messages = [{'role': 'user', 'content': 'What is three plus one?'}]

response = client.chat(
    OLLAMA_MODEL,
    messages=messages,
    tools=[add_numbers],
)

The only difference to a normal query is the additional parameter `tools` which is given a list of helpers appropriate for the query.

Instead of responding with an answer to the query, the LLM will repond with details of what tool to call and the arguments for the call, like follows:

```python
print(response.message.tool_calls)
```
```
[ToolCall(function=Function(name='add_numbers', arguments={'a': 3, 'b': 1}))]
```

Nested in that message is a list of the name(s) of the tool(s) to call and the corresponding arguments.

We then call the tool with the given arguments and feed the result back to the LLM for the final response:

In [ ]:
result_of_toolcall = add_numbers(3, 1)

messages.append({'role': 'tool', 'content': str(result_of_toolcall), 'name': "add_numbers"})

final_response = client.chat(
  OLLAMA_MODEL,
  messages=messages
)

Printing the final response
```
print(final_response.message.content)
```
shuold return something like (the interesting detail is the number 4):
```
The answer to "three plus one" is 4.
```

In [ ]:
print(final_response.message.content)

The only thing missing to fully automate the process is a mapping from tool names to the corresponding functions, and the simplest solution is a dictionary for look-up:

In [ ]:
available_tools = {"add_numbers": add_numbers}

With such a look-up table we can use something like the following to completely automate tool calling:

In [ ]:
def processResponse(messages, original_response, available_tools):
    # If there was no tool call, just return the input
    if not original_response.message.tool_calls:
        return original_response
    # Assume just one tool call
    tool = original_response.message.tool_calls[0]
    tool_name = tool.function.name
    if tool_name not in available_tools:
        print(f"Unknown tool: {tool_name}")
        return original_response
    fcn = available_tools[tool_name]
    mandatory_args = tool.function.arguments
    # Keep optional arguments (if any) separate from mandatory arguments
    # and make sure it is a dictionary
    optional_args = mandatory_args.pop("kwargs", {})
    if not isinstance(optional_args, dict):
        optional_args = {}
    # Call the tool
    result_of_toolcall = fcn(**mandatory_args, **optional_args)
    print(f"Calling tool:\n{tool_name} -> {result_of_toolcall}")
    # Feed the tool result to the LLM
    messages.append(original_response.message)
    messages.append({'role': 'tool', 'content': str(result_of_toolcall), 'name': tool_name})
    print("sending result of tool call back to LLM...")
    # Get final response from model with function outputs
    final_response = client.chat(
      OLLAMA_MODEL,
      messages=messages
    )
    return final_response

Putting it all together:

In [ ]:
messages = [{'role': 'user', 'content': 'What is three plus one?'}]

response = client.chat(
    OLLAMA_MODEL,
    messages=messages,
    tools=[add_numbers],
)

response = processResponse(messages, response, available_tools)

print(response.message.content)

# Calling REST services

However, we are not limited to home grown functions to use as tools, but we can easily make use of e.g. online REST services using modules like `requests`.

In [ ]:
import requests
help (requests.request)

As we can see from the `help` output, the request call provides enough information in its `__doc__` field (try `print(requests.request.__doc__)`) for the LLM to figure out the _schema_, i.e. the semantics of required (and optional) arguments and their types, as well as the expected return value.

We'll use the freely available [meowfacts](https://github.com/wh-iterabb-it/meowfacts) service residing on <https://meowfacts.herokuapp.com/> to fetch random cat facts for the following example.

In [ ]:
messages=[{'role': 'user', 'content': "What can https://meowfacts.herokuapp.com/ tell me about cats?"}]
response = client.chat(
  OLLAMA_MODEL,
  messages=messages,
  tools=[requests.request],
)

```python
print(response.message.tool_calls)
```
```
[ToolCall(function=Function(name='request', arguments={'kwargs': '{"params": {"id": ""}}', 'method': 'GET', 'url': 'https://meowfacts.herokuapp.com/'}))]
```

There are two things to note here:
1. the name of the tool requested by the LLM is the unqualified name `request`, not `requests.request`
2. it got the argument to optional parameter `id` wrong, it is supposed to be numerical id of a specific fact

As an exercise later, try to instruct the LLM (by modifyng the prompt) to follow the REST API [guidelines](https://github.com/wh-iterabb-it/meowfacts/tree/main?tab=readme-ov-file#example-usage).

Putting it all together:

In [ ]:
available_tools = {"request": requests.request}

messages=[{'role': 'user', 'content': "What can https://meowfacts.herokuapp.com/ tell me about cats? Respond in markdown formatted text."}]
response = client.chat(
  OLLAMA_MODEL,
  messages=messages,
  tools=[requests.request],
)

response = processResponse(messages, response, available_tools)

print(response.message.content)

As always, the answer varies from query to query, but you should get a response similar to:

> MeowFacts! That\'s a fun website. According to MeowFacts, here are some interesting and little-known facts about cats:
>
> 1. **Cats have scent glands on their faces**: Cats have scent glands located on either side of their nostrils, as well as on their lips, chin, and near their whiskers.
> 2. **Cats can\'t taste sweetness**: Unlike humans, cats lack the taste receptors for sweetness. This is because they are obligate carnivores and don\'t need to detect sweetness in their diet.
> 3. **A group of cats is called a "clowder"**: Yes, you read that right! A group of cats is officially known as a clowder.
> 4. **Cats have three eyelids**: In addition to their upper and lower eyelids, cats also have a third eyelid called the nictitating membrane or "haw." This helps keep their eyes clean and protected.
> 5. **Cats can jump up to 5 times their own height**: Cats are known for their agility and jumping ability. They can leap impressive distances, thanks to their powerful leg muscles and flexible spines.
> 6. **Cats purr to self-soothe**: While cats often purr when they\'re happy or content, they also purr when they\'re stressed, scared, or even giving birth. Purring is a way for cats to calm themselves down and regulate their breathing.
> 7. **Cats have unique nose prints**: Just like human fingerprints, each cat\'s nose print is unique and can be used to identify them.
> 8. **Cats can sleep for 16 hours a day**: Cats are notorious for their love of sleep. On average, they spend around 16 hours per day snoozing, with some cats sleeping as much as 20 hours in a 24-hour period.
> 9. **Cats can hear sounds that are too faint for humans to detect**: Cats have extremely sensitive hearing and can pick up sounds that are too quiet for humans to detect. They can also hear sounds at higher frequencies than humans can.
> 10. **Cats can\'t see in complete darkness**: While cats have excellent low-light vision, they can\'t see in complete darkness. Their eyes contain a reflective layer called the tapetum lucidum, which helps them see better in low light conditions.
>
> These are just a few of the many fascinating facts about cats that you can find on MeowFacts. Whether you\'re a seasoned cat owner or just a cat enthusiast, there\'s always more to learn about these amazing animals!

# Utilizing portal data

To be written,

---
## Footnotes
<p><small>1. Summarize the practice of 'tool calling' in an LLM query context.</small></p>

---

# Appendix A

In [ ]:
from ollama import Client

class LLM:

    def __init__(self, model):
        self.client = Client(host='10.129.20.4:9090')
        self.model = model

    def prompt(self, query):
        audience = " Your audience is computer scientists, be brief but precise."
        format_info = " Use markdown format for the output."
        response = self.client.chat(model=self.model, messages=[{'role': 'user', 'content': query + audience + format_info}])
        print(response.message.content)
        return response

deepseek = LLM('deepseek-r1:70b')